In [57]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [101]:
text_df = pd.read_csv('./data/fakenews.csv')

In [102]:
text = list(text_df.text.values)
joined_text = ' '.join(text)

In [113]:
partial_text = joined_text[:100000]

In [114]:
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(partial_text.lower())

In [115]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: i for i, token in enumerate(unique_tokens)}

In [116]:
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_words.append(tokens[i + n_words])

In [117]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)


In [118]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_words[i]]] = 1

In [79]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation('softmax'))


In [80]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01), metrics=['accuracy'])
model.fit(X,y, batch_size=128, epochs=40, shuffle=True)

Epoch 1/40
135/135 [==============================] - 35s 226ms/step - loss: 7.0662 - accuracy: 0.0569
Epoch 2/40
135/135 [==============================] - 31s 229ms/step - loss: 6.7185 - accuracy: 0.0575
Epoch 3/40
135/135 [==============================] - 28s 207ms/step - loss: 6.6994 - accuracy: 0.0575
Epoch 4/40
135/135 [==============================] - 26s 195ms/step - loss: 6.6862 - accuracy: 0.0575
Epoch 5/40
135/135 [==============================] - 31s 227ms/step - loss: 6.6748 - accuracy: 0.0575
Epoch 6/40
135/135 [==============================] - 30s 222ms/step - loss: 6.6639 - accuracy: 0.0575
Epoch 7/40
135/135 [==============================] - 25s 189ms/step - loss: 6.6508 - accuracy: 0.0575
Epoch 8/40
135/135 [==============================] - 24s 175ms/step - loss: 6.6401 - accuracy: 0.0575
Epoch 9/40
135/135 [==============================] - 24s 174ms/step - loss: 6.6282 - accuracy: 0.0576
Epoch 10/40
135/135 [==============================] - 24s 174ms/step - l

In [81]:
model.save('./models/nextparaguessing.h5')

e:\final year projects\Para genrator - clg project\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [119]:
model = load_model('./models/nextparaguessing.h5')

In [131]:
def predict_next_words(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
    
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, n_best)[:n_best]

In [134]:
possible = predict_next_words('The president of United states of america announced that he', 5)

1/1 [==============================] - 1s 890ms/step


In [135]:
print([unique_tokens[i] for i in possible])

['sounding', 'brand', 'chanted', 'blamed', 'fundamentalism']


In [136]:
def genrate_text(input_text, text_length, creativity = 3):
    word_sequence = input_text.lower().split()
    current = 0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choise = unique_tokens[random.choice(predict_next_words(sub_sequence, creativity))]
        except:
            choise = random.choice(unique_tokens)
        word_sequence.append(choise)
        current += 1
    return " ".join(word_sequence)

In [137]:
genrate_text("The president of United states of america announced that he", 100, 5)

1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 25ms/step


'the president of united states of america announced that he chanted i thousands deal deal country va history own pandering paris t navy united deal country deal fbi party united brand deal deal country va number group m m va fbi own reprinted series clinton men t deal va country state history brand images thousands deal weapon hatch threshold frail fbi provider hatch country va fbi house television group deal history deal history fbi community united deal deal deal fbi deal deal deal deal country own nationalism m group own nationalism united party country national paris united 94 country separate own country whopping i m in his clinton final meme'